In [1]:
import pandas as pd
import numpy as np
import scipy.integrate
import matplotlib.pyplot as plt
import git
import itertools
from scipy.linalg import svd
from scipy.optimize import least_squares
from datetime import datetime, timedelta
from tqdm.auto import tqdm

## Load data for each county

In [2]:
repo = git.Repo("./", search_parent_directories=True)
homedir = repo.working_dir
datadir = f"{homedir}/data/us/"

In [3]:
# COVID data (fips, county, cases, deaths) columns
df = pd.read_csv(datadir + 'covid/nyt_us_counties.csv')

In [4]:
# population data (FIPS, total_pop, 60plus)
def get_population(fips, pop_df):
    if np.isnan(fips):
        return np.nan
    return pop_df[pop_df['FIPS'] == fips]['total_pop'].values[0]

pop_df = pd.read_csv(datadir + 'demographics/county_populations.csv')
df['population'] = df.apply(lambda row: get_population(row.fips, pop_df), axis=1)
df.loc[df['county'] == 'New York City', 'population'] = 8.399*(10**7)

In [ ]:
# date_processed column 
df['date_processed'] = pd.to_datetime(df['date'].values)
df['date_processed'] = (df['date_processed'] - df['date_processed'].min()) / np.timedelta64(1, 'D')

In [ ]:
df[(df['cases']<10) & (df['deaths']==0) ]

,date,county,state,fips,cases,deaths,population,date_processed
0,2020-01-21,Snohomish,Washington,53061.0,1,0,771904.0,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1,0,771904.0,1.0
2,2020-01-23,Snohomish,Washington,53061.0,1,0,771904.0,2.0
3,2020-01-24,Cook,Illinois,17031.0,1,0,5238541.0,3.0
4,2020-01-24,Snohomish,Washington,53061.0,1,0,771904.0,3.0
5,2020-01-25,Orange,California,6059.0,1,0,3155816.0,4.0
6,2020-01-25,Cook,Illinois,17031.0,1,0,5238541.0,4.0
7,2020-01-25,Snohomish,Washington,53061.0,1,0,771904.0,4.0
8,2020-01-26,Maricopa,Arizona,4013.0,1,0,4155501.0,5.0
9,2020-01-26,Los Angeles,California,6037.0,1,0,10105722.0,5.0


#### Get all required FIPS codes

In [ ]:
sample_sub = pd.read_csv("sample_submission.csv")
fips_list = []
for label in sample_sub['id']:
    fips_list.append(label.split('-')[-1])

## Model: SEIR-QD 
Parameters:  

 $\beta$ = infection rate, from earlier plotting   
 $\delta$ = recovery rate, which we think is on the order of 10-40 days.  
 $\gamma$= transition of exposed individuals to infected, which we aren't sure of, especially with the unknown number of asymptomatics.   
 $\alpha$ = protection rate of susceptible individuals, which we also don't know, and is most likely dynamic over the course of the outbreak.   
 $\lambda$ = transition rate of infected to quarantined with infection, same as above.  
 $\kappa$ = death rate, which we think is around 0.01-0.06. We will leave a range between 0.01 and 0.1.

In [ ]:
def seirqd(dat, t, params, N):
    beta = params[0] / N
    delta = params[1]
    gamma = params[2]
    alpha = params[3]
    lambda_ = params[4]
    kappa = params[5]
    
    s = dat[0]
    e = dat[1]
    i = dat[2]
    q = dat[3]
    r = dat[4]
    d = dat[5]
    sa = dat[6]
    
    dsdt = - beta * s * i - alpha * s
    dedt = beta * s * i - gamma * e
    didt = gamma * e - lambda_ * i
    dqdt = lambda_ * i - delta * q - kappa * q
    drdt = delta * q
    dddt = kappa * q
    dsadt = alpha * s
    
    # susceptible, exposed, infected, quarantined, recovered, died, unsusceptible
    return [dsdt, dedt, didt, dqdt, drdt, dddt, dsadt]

def model_qd(params, data, tmax=-1):
    # initial conditions
    N = data['population'].values[0] # total population
    
    # the parameters are a fraction of the population so multiply by the population
    initial_conditions = N * np.array(params[-5:]) 
    
    # initial conditions
    e0 = initial_conditions[0]
    i0 = initial_conditions[1]
    q0 = initial_conditions[2]
    r0 = initial_conditions[3]
    sa0 = initial_conditions[4]
    
    d0 = data['deaths'].values[0]
    s0 = N - np.sum(initial_conditions) - d0

    yz_0 = np.array([s0, e0, i0, q0, r0, d0, sa0])
    
    # Package parameters into a tuple
    args = (params, N)
    
    n = len(data)
    if tmax > 0:
        n = tmax
    
    # Integrate ODEs
    s = scipy.integrate.odeint(seirqd, yz_0, np.arange(0, n), args=args)

    return s

def fit_leastsq_qd(params, data):
    Ddata = (data['deaths'].values)
    Idata = (data['cases'].values)
    s = model_qd(params, data)

    S = s[:,0]
    E = s[:,1]
    I = s[:,2]
    Q = s[:,3]
    R = s[:,4]
    D = s[:,5]
    SA = s[:,6]
    
    error = np.concatenate((D-Ddata, I - Idata))
    return error

# Helper to return data ever since first min_cases cases
def select_region(df, fips, min_deaths=10):
    d = df.loc[df['fips'] == fips]
    start = np.where(d['deaths'].values >= min_deaths)[0][0]
    d = d[start:]
    return d

#### Actually predict deaths for a county

In [ ]:
# Predict deaths for county
def predict_county(param_guesses, param_ranges, df, fips, min_deaths, predict_days):
    data = select_region(df, fips, min_deaths)
    res = least_squares(fit_leastsq_qd, param_guesses, args=(data,), bounds=np.transpose(np.array(param_ranges)))
    
    # Actual results
    s = model_qd(res.x, data, len(data)+predict_days)
    S = s[:,0]
    E = s[:,1]
    I = s[:,2]
    Q = s[:,3]
    R = s[:,4]
    D = s[:,5]
    SA = s[:,6]
    return D

#### Output formatting helper functions

In [ ]:
def prediction_dates(predictions, fips, start_date, predict_days):
    '''
    Input: (n,) array of predictions, fips code, start_date eg. 04-20-2020, and days to predict 
    Output: (n, 2) array of predictions labeled by date-FIPS
    '''
    start = datetime.strptime(start_date, '%m-%d-%Y')
    data = predictions[len(predictions)-predict_days:]
    result = []
    for i, day_data in enumerate(data):
        label = datetime.strftime(start + timedelta(i), '%m-%d-%Y') + '-' + str(fips)
        dic = {'id':label, '50':day_data}
        result.append(dic)
    return result
    
    
def predict_percentiles(data):
    '''
    Input: table of id, 50-pecentile prediction stored as list of dicts, keys('id', '50')
    Output: table of ids and 10-90 percentiles stored as list of dictionaries
    '''
    output = []
    for row in data:
        label = row['id']
        value = row['50']
        
        # generate samples for percentile
        all_s = []
        samples = 100
        for i in range(samples):
            sample = np.random.normal(loc=value, scale=np.sqrt(value))
            all_s.append(sample)
        
        # row stored as dictionary
        dic = {'id':label}
        for j in range(10, 100, 10):
            dic[str(j)] = np.percentile(all_s, j)
        
        output.append(dic)
    return output

def fill_with_default(start_date, fips, default_val, predict_days):
    '''Return table of date_id with default percentiles, stored as list of dicts'''
    start = datetime.strptime(start_date, '%m-%d-%Y')
    result = []
    for i in range(predict_days):
        label = datetime.strftime(start + timedelta(i), '%m-%d-%Y') + '-' + str(fips)
        dic = {'id':label}
        for j in range(10, 100, 10):
            dic[str(j)] = default_val
        result.append(dic)
    return result
    

## SEIR output for each county
#### Main function

In [ ]:
def generate_output(fips_list, df, start_date, param_guesses=None, param_ranges=None, min_deaths=2, predict_days=14):
    '''
    Input: list of FIPS codes, df with loaded data
    Output: Submission df of 10-90 percentile predictions
    '''
    if param_guesses == None:
        # parameters: beta, delta, gamma, alpha, lambda, kappa
        constants = [2.0, 0.3, 0.2, 0.05, 0.2, 0.03]
        # conditions: E, I, Q, R, SA
        initial_conditions = [0.5e-3, 0.5e-3, 0.3e-3, 0.1e-4, 0.5]
        param_guesses = constants + initial_conditions
    
    if param_ranges == None:
        constants_ranges = [(0.5, 3.0), (0.0, 0.5), (0.0, 0.5), (0.01, 0.5), (0.0, 0.5), (0.005, 0.1)]
        initial_ranges = [(1.0e-7, 0.01), (1.0e-7, 0.01), (1.0e-7, 0.01), (1.0e-7, 0.01), (1.0e-7, 0.9)]
        param_ranges = constants_ranges + initial_ranges
        
    output = []
    counties = df['fips']
    for fips in tqdm(fips_list):
        if fips not in counties:
            output += fill_with_default(start_date, fips, 0, predict_days)
            continue
        
        latest_death = df[df['fips']==fips].iloc[-1]['deaths']
        if latest_death < min_deaths:
            output += fill_with_default(start_date, fips, latest_death, predict_days)
            continue
        
        raw_predict = predict_county(param_guesses, param_ranges, df, fips, min_deaths, predict_days)
        formatted_predict = predict_percentiles(prediction_dates(raw_predict, fips, start_date, predict_days))
        output += formatted_predict
        del raw_predict
        del formatted_predict
    
    columns = ['id'] + [str(j) for j in range(10, 100, 10)]
    return pd.DataFrame(output, columns=columns)


#### Run main function

In [ ]:
final_output = generate_output(fips_list, df, '04-26-2020')

In [ ]:
final_output.to_csv('gorgonio_output.csv')